In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import math

# importação do arquivo bruto de históricos diários de cotações da B3 do ano de 2019 e tratamento dos dados

In [2]:
#importação do arquivo bruto de históricos diários de cotações da B3 do ano de 2019
arquivo_excel = 'COTAHIST_A2019completo.xlsx'

df = pd.read_excel(arquivo_excel, names=['zeroum','ano','mes','dia','momento','id','tipo0','nome','tipo1','tipo2','tipo3','moeda','var0','var1','var2','var3','var4','var5','var6','carac','fim']).astype(str)
df

,zeroum,ano,mes,dia,momento,id,tipo0,nome,tipo1,tipo2,...,moeda,var0,var1,var2,var3,var4,var5,var6,carac,fim
0,1,2019,1,2,2,AAPL34,10,APPLE,DRN,nan,...,R$,6071,6071,5989,6047,6041,5991,6041,0004500000000000001680000000000010159690000000...,BRAAPLBDR004127
1,1,2019,1,2,2,ABBV34,10,ABBVIE,DRN,nan,...,R$,34284,34284,34284,34284,34284,0,0,0000100000000000000030000000000001028520000000...,BRABBVBDR001111
2,1,2019,1,2,2,ABCB4,10,ABC BRASIL,PN,EJS,...,R$,1647,1720,1635,1678,1712,1705,1712,0347700000000000057170000000000095983000000000...,BRABCBACNPR4131
3,1,2019,1,2,2,ABEV3,10,AMBEV S/A,ON,EJ,...,R$,1540,1630,1540,1595,1615,1613,1615,4594100000000001869290000000002982019540000000...,BRABEVACNOR1121
4,1,2019,1,2,2,ADHM3,10,ADVANCED-DH,ON,nan,...,R$,149,150,149,149,150,143,154,0000300000000000000180000000000000026980000000...,BRADHMACNOR9101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779223,1,2019,12,30,82,YDUQM47,80,YDUQE FM,ON,nan,...,R$,152,181,152,160,181,90,0,0000500000000000001040000000000000167320000000...,BRYDUQACNOR3100
779224,1,2019,12,30,82,YDUQM48,80,YDUQE FM,ON,nan,...,R$,192,220,192,209,201,0,0,0000300000000000000320000000000000066990000000...,BRYDUQACNOR3100
779225,1,2019,12,30,82,YDUQM49,80,YDUQE FM,ON,nan,...,R$,258,292,258,282,275,0,0,0000400000000000001050000000000000296400000000...,BRYDUQACNOR3100
779226,1,2019,12,30,82,YDUQN460,80,YDUQE,ON,nan,...,R$,211,211,211,211,211,0,0,0000100000000000000020000000000000004220000000...,BRYDUQACNOR3100


In [3]:
# Criação de coluna de data para definição da série temporal

df['data'] = df[['dia','mes','ano']].agg('/'.join, axis=1)
df['data'] = pd.to_datetime(df['data'])

^
https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
https://stackoverflow.com/questions/17134716/convert-dataframe-column-type-from-string-to-datetime-dd-mm-yyyy-format

In [4]:
#convertendo strings para valores numericos
cols = ['var0','var1','var2','var3','var4','var5','var6']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

^
https://stackoverflow.com/questions/36814100/pandas-to-numeric-for-multiple-columns

In [5]:
#Correção das casas decimais
dfcalc = df[cols].apply(lambda x : x/100)

In [6]:
#Criando coluna apenas com o valor de fechamento
df['close'] = dfcalc.replace(0,np.nan).ffill(axis=1).iloc[:,-1].astype(np.float64)


^
https://stackoverflow.com/questions/53133183/pandas-dataframe-get-value-of-last-nonzero-column

# Filtrando apenas ações do tipo preferenciais e lote padrão para evitar duplicidade de empresas no portfólio

In [7]:
#Criando novo dataframe apenas com os valores necessários para 
colunascsv = ['id','data','tipo0','tipo1','close']
df[colunascsv].loc[lambda df : df['tipo1'] == 'PN',:].to_csv(r'fechamento.csv')
b3_2019 = df[colunascsv].loc[lambda df : df['tipo1'] == 'PN',:]
b3_2019 = b3_2019.loc[lambda b3_2019 : b3_2019['tipo0'] == '10']
#b3_2019 = b3_2019.drop(b3_2019[b3_2019.tipo0 != 20]).index
b3_2019

,id,data,tipo0,tipo1,close
2,ABCB4,2019-02-01,10,PN,17.12
8,ALPA4,2019-02-01,10,PN,17.50
11,ALUP4,2019-02-01,10,PN,5.80
21,AZEV4,2019-02-01,10,PN,1.02
22,AZUL4,2019-02-01,10,PN,36.35
...,...,...,...,...,...
231265,SLED4,2019-12-30,10,PN,2.14
231267,TCNO4,2019-12-30,10,PN,1.41
231268,TEKA4,2019-12-30,10,PN,6.79
231270,BPAN10,2019-12-30,10,PN,10.90


# agrupando tabela por dia versus empresa e calculando as variações diárias

In [8]:
b3diff = b3_2019.sort_values(['id','data'])
b3diff.loc[b3diff['id'] == b3diff['id'].shift(), 'logret'] = \
    np.log(b3diff['close']) - np.log(b3diff['close'].shift())


b3agr = b3diff.groupby(['data','id'])['logret'].last().unstack().fillna(0)
b3agr

id,ABCB10,ABCB4,ALPA4,ALUP4,AZEV4,AZUL4,BALM4,BAUH4,BBDC4,BDLL4,...,TEKA4,TELB4,TIET4,TKNO4,TOYB4,TRPL4,TXRX4,VIVT4,WHRL4,WLMM4
data,,,,,,,,,,,,,,,,,,,,,
2019-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-01-03,0.000000,-0.077387,-0.009980,0.019364,-0.044060,-0.010539,0.000000,0.088947,-0.049294,0.000000,...,0.000000,-0.068286,-0.012903,0.0,0.000000,0.009602,0.137870,-0.067951,-0.148009,0.001561
2019-01-04,0.070455,0.065154,-0.255119,0.077722,-0.114410,0.056509,0.000000,0.156458,-0.165218,0.000000,...,-0.063422,0.017514,-0.035246,0.0,-0.048527,-0.025340,0.000000,0.032874,0.112478,0.086621
2019-01-07,0.000000,-0.025975,0.288652,0.074470,0.688084,0.078956,0.108509,0.297459,0.042340,-0.282232,...,-0.120018,-0.076086,0.056660,0.0,0.109771,-1.146423,-0.095310,0.026729,0.057867,-0.058927
2019-01-08,0.000000,-0.070874,0.025851,-0.019003,0.205734,0.250154,0.000000,0.114776,-0.094115,-0.140762,...,0.120018,0.022231,0.045557,0.0,0.213193,-0.003659,0.024693,0.069688,0.052798,0.129350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-20,0.000000,0.006518,-0.011372,-0.005831,-0.076161,-0.012379,-0.032563,0.001325,-0.021071,-0.014775,...,0.007273,-0.020549,0.000000,0.0,0.000000,-0.000445,0.000000,0.010595,0.074533,-0.057432
2019-12-23,0.000000,0.006476,0.010761,0.004667,0.195425,0.011859,0.003005,0.000000,0.006145,0.073044,...,-0.016070,-0.012535,0.013158,0.0,0.000000,0.011074,0.062848,0.008601,0.098820,0.057432
2019-12-26,0.000000,0.001488,0.012158,0.046626,0.297252,-0.009231,0.000000,-0.006645,0.012453,-0.006309,...,0.016070,0.019432,0.006515,0.0,0.000000,0.011389,-0.012772,0.012934,0.081658,0.019194


# Cálculo do índice de sharpe. O ativo livre de risco considerado foi o Tesouro Prefixado 2019 que previsa retorno anual de 6,54% (0,0251% ao dia)

In [9]:
#novo dataframe com o indice de sharpe calculado
dfsort_std = b3agr.clip(upper=0)
dfsort_std

dfratios = b3agr

dfratios.loc['mean'] = b3agr.mean()
dfratios.loc['std'] = b3agr.std()
dfratios.loc['std_sort'] = dfsort_std.std()
dfratios.loc['sharpe'] = (lambda x,y : math.sqrt(252)*((x - (0.000251421742025526)) / y))(dfratios.loc['mean',:],dfratios.loc['std',:])
dfratios.loc['sortino'] = (lambda x,y : math.sqrt(252)*((x - (0.000251421742025526)) / y))(dfratios.loc['mean',:],dfratios.loc['std_sort',:])
dfratios.to_csv(r'indices.csv')
dfratios

id,ABCB10,ABCB4,ALPA4,ALUP4,AZEV4,AZUL4,BALM4,BAUH4,BBDC4,BDLL4,...,TEKA4,TELB4,TIET4,TKNO4,TOYB4,TRPL4,TXRX4,VIVT4,WHRL4,WLMM4
data,,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-01-03 00:00:00,0.000000,-0.077387,-0.009980,0.019364,-0.044060,-0.010539,0.000000,0.088947,-0.049294,0.000000,...,0.000000,-0.068286,-0.012903,0.000000,0.000000,0.009602,0.137870,-0.067951,-0.148009,0.001561
2019-01-04 00:00:00,0.070455,0.065154,-0.255119,0.077722,-0.114410,0.056509,0.000000,0.156458,-0.165218,0.000000,...,-0.063422,0.017514,-0.035246,0.000000,-0.048527,-0.025340,0.000000,0.032874,0.112478,0.086621
2019-01-07 00:00:00,0.000000,-0.025975,0.288652,0.074470,0.688084,0.078956,0.108509,0.297459,0.042340,-0.282232,...,-0.120018,-0.076086,0.056660,0.000000,0.109771,-1.146423,-0.095310,0.026729,0.057867,-0.058927
2019-01-08 00:00:00,0.000000,-0.070874,0.025851,-0.019003,0.205734,0.250154,0.000000,0.114776,-0.094115,-0.140762,...,0.120018,0.022231,0.045557,0.000000,0.213193,-0.003659,0.024693,0.069688,0.052798,0.129350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mean,0.000137,0.000091,0.002015,0.000720,0.005466,0.001810,-0.001082,0.005196,-0.000853,-0.002357,...,-0.001114,-0.000096,0.000988,0.000334,0.000480,-0.005020,0.002046,0.000654,0.003001,0.002943
std,0.010094,0.038171,0.142240,0.146396,0.360759,0.102049,0.068522,0.285944,0.074767,0.210064,...,0.070625,0.137416,0.052399,0.024770,0.040138,0.383024,0.196674,0.042966,0.126462,0.166023
std_sort,0.006352,0.023605,0.109700,0.098834,0.200649,0.074311,0.050303,0.232536,0.048118,0.106669,...,0.043573,0.077154,0.036199,0.017862,0.029779,0.279235,0.141039,0.031803,0.086874,0.123203


# Cálculo da otimização utilizando Sortino - cálculo dos pesos de cada ativo em um portfólio dos 20 melhores índices

Restrições aplicadas aos pesos
1 - Deve ser maior que 0,01;
2 - Deve ser menor que 0,15;
3 - Deve somar 1;

In [10]:
import cvxpy as cp
from cplex import *
from operator import mul

In [11]:
w = cp.Variable(10)
assets_sortino = dfratios.loc['sortino',:].nlargest(10).reset_index()

print(assets_sortino)

risk = assets_sortino['sortino'].fillna(0).to_numpy()

restricoes = [
    w >= 0.03,
    w <= 0.25,
    cp.sum(w) == 1
]

objetivo = cp.Maximize(
        cp.sum(
            cp.multiply(risk,w)))

prob = cp.Problem(objetivo,restricoes)

prob.solve(cp.CPLEX, verbose = True)

print (w.value)

retornosort = dfratios[assets_sortino['id']].loc[['mean','std_sort','sortino'],:]
retornosort

       id   sortino
0  PINE10  2.983979
1   EEEL4  1.071178
2   BMGB4  0.843404
3   TASA4  0.835528
4   CSAB4  0.800110
5  BPAN10  0.770751
6   EMAE4  0.766643
7   MSPA4  0.713280
8   EKTR4  0.549469
9   WHRL4  0.502384
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_QCPDuals                  2
Tried aggregator 1 time.
LP Presolve eliminated 20 rows and 0 columns.
Reduced LP has 1 rows, 10 columns, and 10 nonzeros.
Presolve time = 0.02 sec. (0.01 ticks)
Initializing dual steep norms . . .
[0.25 0.25 0.25 0.07 0.03 0.03 0.03 0.03 0.03 0.03]


id,PINE10,EEEL4,BMGB4,TASA4,CSAB4,BPAN10,EMAE4,MSPA4,EKTR4,WHRL4
data,,,,,,,,,,
mean,0.000567,0.002878,0.000518,0.000935,0.001555,0.001123,0.002473,0.001283,0.001111,0.003001
std_sort,0.001681,0.038926,0.005015,0.012993,0.025865,0.017955,0.046000,0.022960,0.024825,0.086874
sortino,2.983979,1.071178,0.843404,0.835528,0.800110,0.770751,0.766643,0.713280,0.549469,0.502384


# # Cálculo da otimização utilizando Sharpe - cálculo dos pesos de cada ativo em um portfólio dos 20 melhores índices

Restrições aplicadas aos pesos
1 - Deve ser maior que 0,01;
2 - Deve ser menor que 0,15;
3 - Deve somar 1;

In [12]:
z = cp.Variable(10)
assets_sharpe = dfratios.loc['sharpe',:].nlargest(10).reset_index()

print(assets_sharpe)

risk = assets_sharpe['sharpe'].fillna(0).to_numpy()

restricoes = [
    z >= 0.03,
    z <= 0.25,
    cp.sum(z) == 1
]

objetivo = cp.Maximize(
        cp.sum(
            cp.multiply(risk,z)))

prob = cp.Problem(objetivo,restricoes)

prob.solve(cp.CPLEX, verbose = True)

print (z.value)

retornosharpe = dfratios[assets_sharpe['id']].loc[['mean','std','sharpe'],:]
retornosharpe

       id    sharpe
0  PINE10  0.862355
1   EEEL4  0.677476
2   EMAE4  0.545916
3  BPAN10  0.506616
4   TASA4  0.490866
5   BMGB4  0.446363
6   CSAB4  0.426279
7   MSPA4  0.380685
8   EKTR4  0.354975
9   WHRL4  0.345117
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_QCPDuals                  2
Tried aggregator 1 time.
LP Presolve eliminated 20 rows and 0 columns.
Reduced LP has 1 rows, 10 columns, and 10 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)
Initializing dual steep norms . . .
[0.25 0.25 0.25 0.07 0.03 0.03 0.03 0.03 0.03 0.03]


id,PINE10,EEEL4,EMAE4,BPAN10,TASA4,BMGB4,CSAB4,MSPA4,EKTR4,WHRL4
data,,,,,,,,,,
mean,0.000567,0.002878,0.002473,0.001123,0.000935,0.000518,0.001555,0.001283,0.001111,0.003001
std,0.005817,0.061548,0.064599,0.027316,0.022115,0.009477,0.048548,0.043020,0.038426,0.126462
sharpe,0.862355,0.677476,0.545916,0.506616,0.490866,0.446363,0.426279,0.380685,0.354975,0.345117


# Retornos anualizados dos dois portfólios

In [29]:
print("O retorno anual do portfólio calculado por Sortino é: ", math.exp(sum(map(mul, retornosort.loc['mean',:], w.value))*252)-1)
print("O retorno anual do portfólio calculado por Sharpe é:", math.exp(sum(map(mul, retornosharpe.loc['mean',:], z.value))*252)-1)

O retorno anual do portfólio calculado por Sortino é:  0.4132859084809497
O retorno anual do portfólio calculado por Sharpe é: 0.5780661081338336
